# Défi Grosses Data 2018
Soizick

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import Annex

In [12]:
pd.options.mode.chained_assignment = None

## Sans les données manquantes

In [13]:
X_train,X_test,Y_train,Y_test,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.3,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='drop')

We are addressing your request.
Data has been imported. Size: (189280, 31)
Months converted to int.
Dummies added.
Date dropped.
26528 data points deleted. 14.02 %
Train size: 113926, Test size: 48826
Data scaled


## PLS

In [14]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

#### 1. Test en prennant `n_components=30`

In [15]:
pls = PLSRegression(n_components=30,max_iter=1000)
pls.fit(X_train,Y_train)

PLSRegression(copy=True, max_iter=1000, n_components=30, scale=True,
       tol=1e-06)

In [16]:
Y_pred = pls.predict(X_test)

In [17]:
print("MSE =",mean_squared_error(Y_test,Y_pred))

MSE = 1.53042014772


#### 2. Validation croisée

In [19]:
param=[{"n_components":list(range(2,30,1))}]
pls1 = GridSearchCV(PLSRegression(), param,cv=5,n_jobs=-1)
pls1_Opt=pls1.fit(X_train, Y_train)

In [20]:
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - pls1_Opt.best_score_,pls1_Opt.best_params_))

Meilleur score = 0.039857, Meilleur paramètre = {'n_components': 24}


On prend 24 composantes

In [21]:
# Estimation de l'erreur de prévision sur l'échantillon test
1-pls1_Opt.score(X_test,Y_test)

0.038982910604067222

In [22]:
y_chap = pls1_Opt.predict(X_test)

In [23]:
print("MSE =",mean_squared_error(Y_test,y_chap))

MSE = 1.53042038199
